<a href="https://colab.research.google.com/github/abzer005/Summer-School_Functional-Metabolomics/blob/main/20220715_Data_CleanUp_SummerSchool_Metabolomics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Clean up
Authors: Abzer Kelminal (abzer.shah@uni-tuebingen.de) <br>
Edited by:  <br>
Input file format: .csv files or .txt files <br>
Outputs: .csv files  <br>
Dependencies: ggplot2, dplyr

In [40]:
#R Google Colab currently runs on the version ''4.2.0'. Hence, we are checking which R version we have:
R.version.string

[1] "R version 4.2.0 (2022-04-22)"

In [ ]:
#installing and calling the necessary packages:
if (!require("ggplot2")) install.packages("ggplot2")
if (!require("dplyr")) install.packages("dplyr")

##Mounting Google Drive:

In [ ]:
install.packages('googledrive') #only need to install occasionally
install.packages('httpuv') 
library('googledrive') 
library('httpuv')

In [ ]:
if (file.exists("/usr/local/lib/python3.7/dist-packages/google/colab/_ipython.py")) { #may update python version occasionally
  install.packages("R.utils")
  library("R.utils")
  library("httr")
  my_check <- function() {return(TRUE)}
  reassignInPackage("is_interactive", pkgName = "httr", my_check) 
  options(rlang_interactive=TRUE)
}

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [37]:
drive_auth(use_oob = TRUE, cache = FALSE)

Please point your browser to the following url: 

https://accounts.google.com/o/oauth2/auth?client_id=603366585132-dpeg5tt0et3go5of2374d83ifevk5086.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&login_hint=abzer.kelmina%40gmail.com



Enter authorization code: 4/1AdQt8qgLywP7lSuy-Baa3QSRLtV4tbC-qJbPy5s4fRLBw5IbxRtYdHVpJbA


In [38]:
getwd()

[1] "/content"

In [48]:
drive_ls('Files for Summer School_Xenobiotic metabolism')$name

[1] "MZmine Files"                                
[2] "GNPS Files"                                  
[3] "Data CleanUp Jupyter Notebook output"        
[4] "Xenobiotic_Metabolism_metadata.txt"          
[5] "Data_CleanUp_SummerSchool_Metabolomics.ipynb"

In [54]:
x <- drive_get("~/Files for Summer School_Xenobiotic metabolism/MZmine Files/Xenobiotic_Metabolism_ChemProp2_GapFilled_QuantTable.csv")
drive_download(x)
z <- read.csv("/content/Xenobiotic_Metabolism_ChemProp2_GapFilled_QuantTable.csv")
head(z)





→ Files retrieved so far: 100

✔ The input `path` resolved to exactly 1 file.



ERROR: ignored

In [52]:
library(readr)

In [56]:
md <- drive_get("~/Files for Summer School_Xenobiotic metabolism/MZmine Files/Xenobiotic_Metabolism_ChemProp2_GapFilled_QuantTable.csv")
drive_download(md)
md <- read.csv('/content/Xenobiotic_Metabolism_metadata.txt',sep='\t') 
head(md)

Warning message in file(file, "rt"):
“cannot open file '/content/Xenobiotic_Metabolism_metadata.txt': No such file or directory”


ERROR: ignored

## Reading the input data using URL (from Github):

In [ ]:
## Non-gap filled
nft_url <- 'https://raw.githubusercontent.com/madeleineernst/Metabolomics_SummerSchool_2022/main/data/MZmine/Xenobiotic_Metabolism_ChemProp2_NonGapFilled_QuantTable.csv'
## Gap filled
ft_url <- 'https://raw.githubusercontent.com/madeleineernst/Metabolomics_SummerSchool_2022/main/data/MZmine/Xenobiotic_Metabolism_ChemProp2_GapFilled_QuantTable.csv'
md_url <- 'https://raw.githubusercontent.com/madeleineernst/Metabolomics_SummerSchool_2022/main/data/Xenobiotic_Metabolism_metadata.txt'

In [ ]:
nft <- read.csv(nft_url, header = T, check.names = F)
ft <- read.csv(ft_url, header = T, check.names = F)
md <- read.csv(md_url, header = T, check.names = F, sep = '\t')

## Setting a local working directory and creating an automatic result directory:

In [ ]:
# setting the current directory as the working directory
Directory <- normalizePath(readline("Enter the path of the folder with input files: "),"/",mustWork=FALSE)
setwd(Directory)

In [ ]:
getwd() #to get the working directory 

In [ ]:
# Getting all the files in the folder
dirs <- dir(path=paste(getwd(), sep=""), full.names=TRUE, recursive=TRUE)
folders <- unique(dirname(dirs))
files <- list.files(folders, full.names=TRUE)
files_1 <- basename((files))
files_2 <- dirname((files))
# Creating a Result folder
dir.create(path=paste(files_2[[1]], "_Results", sep=""), showWarnings = TRUE)
fName <-paste(files_2[[1]], "_Results", sep="")

**<font color='red'> In the following line, enter the required file ID numbers separated by commas. For example as: 1,2,3 </font>**

In [ ]:
print(files_1)
input <- as.double(unlist(strsplit(readline("Specify the ID numbers of feature-file, metadata:"), split=",")))

#Gets the extension of each file. Ex:csv
pattern <- c()
for (i in files_1){
  sep_file <- substr(i, nchar(i)-2,nchar(i))
  pattern <- rbind(pattern,sep_file)
}
#pattern

ft <- read.csv(files_1[input[1]],sep = ifelse(pattern[input[1]]!="csv","\t",","), header=TRUE,check.names = FALSE) # By applying 'row.names = 1', the 1st column 'ID' becomes the row names
md <-read.csv(files_1[input[2]], sep = ifelse(pattern[input[2]]!="csv","\t",","), header=TRUE,check.names = FALSE)

In [ ]:
#If you have cluster_info file:
if(readline("Do you have non gap-filled feature table? Y/N:")=="Y"){
  nft <- as.double(readline("Enter the ID number of non-gap-filled feature file:"))
  nft<- read.csv(files_1[nft],sep=ifelse(pattern[nft]!="csv","\t",","), header = TRUE,check.names = FALSE)
}

Lets check if the data has been read correclty!!

In [ ]:
head(ft)
dim(ft)

In [ ]:
head(nft)
dim(nft)

In [ ]:
head(md)
dim(md)

Trying to bring the feature table and metadata in the correct format:

In [ ]:
#Removing Peak area extensions
colnames(ft) <- gsub(' Peak area','',colnames(ft))
colnames(nft) <- gsub(' Peak area','',colnames(nft))
md$filename<- gsub(' Peak area','',md$filename)

#Removing if any NA columns present in the md file
md <- md[,colSums(is.na(md))<nrow(md)]

#Changing the row names of the files
rownames(md) <- md$filename
md <- md[,-1]
rownames(ft) <- paste(ft$'row ID',round(ft$'row m/z',digits = 3),round(ft$'row retention time',digits = 3), sep = '_')
rownames(nft) <- paste(nft$'row ID',round(nft$'row m/z',digits = 3),round(nft$'row retention time',digits = 3), sep = '_')

#Picking only the files with column names containing 'mzML'
ft <- ft[,grep('mzML',colnames(ft))]
nft <- nft[,grep('mzML',colnames(nft))]

# Converting replicate attributes into factors (categorical data)
md$ATTRIBUTE_replicates <- as.factor(md$ATTRIBUTE_replicates)

Lets check the files once again!!

In [ ]:
head(nft)
dim(nft)

In [ ]:
head(ft)
dim(ft)

In [ ]:
head(md)
dim(md)

### Creating a function named FrequencyPlot:  
The below function takes in the two input datatables: for example, gapfilled and non-gapfilled, calculates the frequency distribution of the data in the order of 10 and produces a grouped barplot showing the distribution as output. The frequency plot shows where the features are present in higher number.

In [ ]:
#'Global' settings for plot size in the output cell
options(repr.plot.width=10, repr.plot.height=8,res=600) #For google collab
#options(repr.plot.width=5, repr.plot.height=3) #For Jupyter Notebook

In [ ]:
FrequencyPlot <- function(x1,x2){
  
   #creating bins from -1 to 10^10 using sequence function seq()
    bins <- c(-1,0,(1 * 10^(seq(0,10,1)))) 
    
    #cut function cuts the give table into its appropriate bins
    scores_x1 <- cut(as.matrix(x1),bins,labels = c('0','1','10','1E2','1E3','1E4','1E5','1E6','1E7','1E8','1E9','1E10')) 
    
    #transform function convert the tables into a column format: easy for visualization 
    Table_x1<-transform(table(scores_x1)) #contains 2 columns: "scores_x1", "Freq"
    
    #Repeating the same steps for x2
    scores_x2 <- cut(as.matrix(x2),bins,labels = c('0','1','10','1E2','1E3','1E4','1E5','1E6','1E7','1E8','1E9','1E10'))
    Table_x2<-transform(table(scores_x2))
  
    #Getting the names of x1 and x2
    arg1 <- deparse(substitute(x1))
    arg2 <-deparse(substitute(x2))
    
    #Creating a data frame for plotting
    data_plot <- as.data.frame(c(Table_x1$Freq,Table_x2$Freq)) #Concatenating the frequency info of both tables rowwise
    colnames(data_plot) <- "Freq" #naming the 1st column as 'Freq'
    data_plot$Condition <- c(rep(arg1,12),rep(arg2,12)) #adding a 2nd column 'Condition', which just repeats the name of x1 and x2 accordingly
    data_plot$Range_bins <- rep(Table_x1$scores_x1,2) #Adding 3rd column 'Range Bins'
    data_plot$Log_Freq <- log(data_plot$Freq+1) #Log scaling the frequency values
    
    ## GGPLOT2
    BarPlot <- ggplot(data_plot, aes(Range_bins, Log_Freq, fill = Condition)) + 
    geom_bar(stat="identity", position = "dodge", width=0.4) + 
    scale_fill_brewer(palette = "Set1") +
    ggtitle(label="Frequency plot") +
    xlab("Range") + ylab("(Log)Frequency") + labs(fill = "Data Type") + 
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +   # setting the angle for the x label
    theme(axis.text.y = element_text(angle = 45, vjust = 0.5, hjust=1)) +   # setting the angle for the y label
    theme(plot.title = element_text(hjust = 0.5)) # centering the plot title
  
    print(BarPlot)
}  

**About the experiment:**
- Bacteria (B.subtilis and E.coli) were treated with a pool of antibiotics (Sulfamethoxazole, sulfadimethoxine, cyproconazole) including a herbicide Asulam, taken at a concentration lower than their MIC (minimum inhibitory concentration).
- The samples were collected at different timepoints, the compounds were extracted (with 50% EtOAc) and measured using LC-MS/MS.
- The goal of the experiment was to look for any potential biotransformation. eg: Drug or xenobiotic metabolism

## Blank Removal:

(Note: In LC-MS/MS, we use solvents also called as Blanks which are usually injected time-to-time to prevent carryover of the sample) </br>

For the Blank removal step, we need to split the data as control and samples. </br>

**The blanks we are referring to here, is the control blanks in the experiment and not the LC-MS/MS blanks.**
- The control blanks here is the sample without treatment. 
- Samples are biological replicates with treatment and we have two sets of data: B.sub and E.coli. </br>

In general, having multiple control blanks helps us to compare any variation in the data. Comparing control to the sample helps us to identify the background features that contribute to any technical variation. A common filtering method is to use a cutoff to remove features that are not present sufficient enough in our biological samples.

1. We find an average for all the feature intensities in your control set and sample set.
Therefore, for n no.of features in a control or sample set, we get n no.of averaged features.
2. Next, we get a ratio of this average_control vs average_sample. This ratio Control/sample tells us how much of that particular feature of a sample gets its contribution from control. If it is more than 30% (or Cutoff as 0.3), we consider the feature as noise.
3. The resultant information (if ratio > Cutoff or not) is stored in a bin
4. We count the no.of features in the bin that satisfies the condition ratio > cutoff, and consider those features as 'noise or background features'.
5. We also try to visualize the frequency distribution in our data Ctrl and samples

For a dataset containing several batches, the filtering steps are performed batch-wise.

In [ ]:
input_data <-ft

In [ ]:
Ctrl <-  input_data[,grep('_C',colnames(input_data))]
Samples <- input_data[,-grep('_C',colnames(input_data))]

dim(Ctrl)
dim(Samples)

### Similarly, we can also subset the data belonging to only one batch, for ex:"B.subtilis", for further analysis.

In [ ]:
head(md)
print(matrix(data=colnames(md),nrow=length(colnames(md))))

In [ ]:
Condition <- as.double(unlist(strsplit(readline("Enter the IDs of interested attributes separated by commas:"),split=",")))

In [ ]:
#Specifying which conditions to keep under the selected attributes

Meta_Filtered <-md
for(i in 1:length(Condition)){
    
    #Shows the different levels within each selected condition:
    Levels_Cdtn <- levels(as.factor(Meta_Filtered[,Condition[i]]))
    print(matrix(Levels_Cdtn,length(Levels_Cdtn)))
    
    #These lines are not needed in R console, but in Jupyter Notebook to get the previous print statement working
    flush.console()  
    Sys.sleep(0.2)
    
    #Among the shown levels of ana ttribute, select the ones to keep
    Cdtn <- as.double(unlist(strsplit(readline("Enter the IDs of condition(s) you want to KEEP (separated by commas):"), split=',')))
    Levels_Cdtn[Cdtn]
    
    #Selecting only rows in meta_filtered that match the condition
    Meta_Filtered <- Meta_Filtered[(Meta_Filtered[,Condition[i]] == Levels_Cdtn[Cdtn]),]
  }

#Removing all the rows with only NAs
Meta_Condition <- subset(Meta_Filtered,rowSums(is.na(Meta_Filtered))!=ncol(Meta_Filtered))

In [ ]:
head(Meta_Condition)
dim(Meta_Condition)

In [ ]:
#Picking only the columns in ft that are same as rownames in final metadata
ft_Condition <- input_data[,which(colnames(input_data)%in%rownames(Meta_Final))] 

Now we can continue splitting the batch data into control and sample and proceed with next steps:

In [ ]:
input_data<- ft_Condition
Ctrl <-  input_data[,grep('_C',colnames(input_data))]
Samples <- input_data[,-grep('_C',colnames(input_data))]

dim(Ctrl)
dim(Samples)

<font color='red'>Note: The Cutoff value chosen for getting the files in the folder **Preprocessing Jupyter Output files-Data CleanUp** in the github: 0.9 </font>

In [ ]:
if(readline('Do you want to perform Blank Removal- Y/N:')=='Y'){
    
    #When cutoff is low, more noise (or background) detected; With higher cutoff, less background detected, thus more features observed
    Cutoff <- as.numeric(readline('Enter Cutoff value between 0.1 & 1:')) # (i.e. 10% - 100%). Ideal cutoff range: 0.1-0.3
    
    #Getting mean for every feature in Ctrl and Samples
    Avg_ctrl <- rowMeans(Ctrl, na.rm= FALSE, dims = 1) # set na.rm = FALSE to check if there are NA values. When set as TRUE, NA values are changed to 0
    Avg_samples <- rowMeans(Samples, na.rm= FALSE, dims = 1)
    
    #Getting the ratio of Ctrl vs Sample
    Ratio_Ctrl_Sample <- (Avg_ctrl+1)/(Avg_samples+1)
    
    # Creating a bin with 1s when the ratio>Cutoff, else put 0s
    Bg_bin <- ifelse(Ratio_Ctrl_Sample > Cutoff, 1, 0 )


    # Checking if there are any NA values present. Having NA values in the 4 variables will affect the final dataset to be created
    temp_NA_Count <-cbind(Avg_ctrl ,Avg_samples,Ratio_Ctrl_Sample,Bg_bin)
    
    print('No of NA values in the following columns:')
    print(colSums(is.na(temp_NA_Count)))

     #Calculating the number of background features and features present
    Bg_Features <- sum(Bg_bin ==1,na.rm = TRUE)
    No_of_Features <- nrow(input_data) - Bg_Features
    
    print(paste("No.of Background or noise features:",Bg_Features))
    print(paste("No.of features after excluding noise:",No_of_Features)) 

    input_data1 <- cbind(as.matrix(input_data),Bg_bin)    
    plot_CtrlSample <- FrequencyPlot(Samples,Ctrl)
}

**TO NOTE:** We have not yet removed the blanks. We have just calculated the noise features according to our cutoff and these will be eliminated in the later steps.

### Imputation: 

For several reasons, real world datasets might have some missing values in it, in the form of NA, NANs or 0s. Eventhough the gapfilling step of MZmine fills the missing values, we still end up with some missing values or 0s in our feature table. This could be problematic for statistical analysis. 
In order to have a better dataset, we cannot simply discard those rows or columns with missing values as we will lose a chunk of our valuable data.
Instead we can try imputing those missing values. Imputation involves replacing the missing values in the data with a meaningful, reasonable guess. There are several methods, such as:  
1) Mean imputation (replacing the missing values in a column with the mean or average of the column)  
2) Replacing it with the most frequent value  
3) Several other machine learning imputation methods such as k-nearest neighbors algorithm(k-NN), Hidden Markov Model(HMM)

One such method, we are going to use is: **to replace the zeros from the gapfilled quant table with the non-gap filled table** we get from MZmine. In order to do that, we can visualize our data distribution using the frequenct plot

In [ ]:
GapFilled <- input_data
NotGapFilled <- nft

In [ ]:
if(readline('Do you want to perform Imputation with minimum value of NonGapFilled table? - Y/N:')=='Y'){
    
    plot<- FrequencyPlot(GapFilled,NotGapFilled)
    
    Arg1 = plot$data$Condition[1]
    Arg2 = plot$data$Condition[13]
    
    # accessing the datatable of plot and subsetting with the condition: Eliminating the Range (or bin) 0 and Ranges with zero frequencies 
    plotData_New <- subset(plot$data,plot$data$Freq!=0 & plot$data$Range_bins !=0) 
    
    #getting the first appearing value of this new plot datatable
    First_val_temp <- aggregate(plotData_New$Freq, by=list(plotData_New$Condition), FUN=first) 
    
    # Subsetting the rows in the plotData_New that has the first appearing values
    First_val <- plotData_New[plotData_New$Freq %in% c(First_val_temp$x[1],First_val_temp$x[2]),]
  
    print(paste0("The Range with a minimum value greater than 0 for ",Arg1,":", First_val$Range_bins[1]))
    print(paste0("The Range with a minimum value greater than 0 for ",Arg2,":", First_val$Range_bins[2]))
    
    # getting the 2nd minimum value of non-gap filled data. (The first minimum value in the data table is usually zero)
    RawLOD <- round(min(NotGapFilled[NotGapFilled!=min(NotGapFilled)])) 
    
    print(paste0("The minimum value in the Non-gap filled data other than 0 : ",RawLOD))
    GapFilled[GapFilled==0 | GapFilled<RawLOD] <- RawLOD # Replacing zeros in the gap-filled file as well as values<RawLOD with RawLOD
    RawLOD_Table <- GapFilled
    
    #write.csv(RawLOD_Table, file=paste0('Quant_Table_filled_with_MinValue_',RawLOD,'_NotGapFilled','.csv'),row.names =FALSE) 
    
    input_data <- GapFilled
} else input_data <- GapFilled

### Imputation with a Cutoff LOD:
Previously we have replaced the zeros in our ft(gapfilled) with the minimum value in nft(non gapfilled). Ex: 3766.<br>

Instead, we can also only use ft and see the frquency distribution of its features with the frequency plot. The frequency plot shows where the features are present in higher number.

For ex: If until range 10-100, (shown in the figure as 1E2) there are no or very less features, we want to exclude until that range and consider from range (100-1000), or, in other words, '1E3' or '1000' as Cutoff_LOD. This value will be used to replace the zeros in the data table.

The following step asks if the imputation was already performed, if so, it takes that value as the Cutoff_LOD, else, we get to specify our Cutoff_LOD based on the frequency plot.
Once we have our Cutoff_LOD:
- We create  a temporary dataset with all the feature intensites of your sample (not the ctrl, only the sample) and checking it against the cutoff_LOD value. If it is less than the cutoff_LOD, we replace it with cutoff_LOD. Thus, for instance, if we take 1000 as cutoff_LOD, our sample data will be filled with a bunch of 1000s now instead of zeros.
- Then we create a Final dataset using the temporary dataset. Here, we try to see if each feature from all samples is noise or not (from step 4 under Blank Removal), if it noise, we replace the feature with cutoff_LOD, else we keep the info from the temporary dataset.


In [ ]:
Cutoff_LOD <-ifelse(readline("Was Imputation step already performed? Y/N :")=="Y",RawLOD,as.numeric(readline("Enter your Cutoff LOD here:")))  #Enter the LOD value as seen in the frequency plot
temp_matrix <- c()
for (i in 1:ncol(Samples)){ 
    
    #Replacing the Sample intensities with Cutoff_LOD, if they are lower than Cutoff_LOD
    x <- ifelse(Samples[,i] > Cutoff_LOD, Samples[,i],Cutoff_LOD)
    temp_matrix <- cbind(temp_matrix,as.matrix(x))
}
colnames(temp_matrix) <- colnames(Samples)
  
Final_matrix <-c()
for (i in 1:ncol(temp_matrix)){
    
     #Replacing the feature row with Cutoff_LOD if its Bg_bin value is 1, indicating it as noise, else retain the temporary dataset as such.
    x <-ifelse(input_data1[,ncol(input_data1)] ==1, Cutoff_LOD, temp_matrix[,i])
    Final_matrix <-cbind(Final_matrix,x)
}
colnames(Final_matrix) <- colnames(Samples)

In [ ]:
write.csv(Final_matrix,file.path(fName, paste0('Processed_QuantTable_filled_with_',Cutoff_LOD,'_CutOff_Used_',Cutoff,'_Bsub','.csv')),row.names =TRUE)

In [ ]:
head(Final_matrix)

In [ ]:
#removing all the rows with only cutoff values:
#Final_matrix<-Final_matrix[rowMeans(Final_matrix)!= Cutoff_LOD,]  

## Normalization:
The following code performs sample-centric (column-wise) normalisation:

In [ ]:
if (readline("Do you want to perform Normalization: Y/N:") == 'Y'){
    input_data <- Final_matrix
    
    #Getting column-wise sums of the input-data
    sample_sum <- colSums(input_data, na.rm= TRUE, dims = 1)
    
    #Dividing each element of a particular column with its column sum
    Normalized_data <- c()
    for (i in 1:ncol(input_data)){
        x <- input_data[,i] / sample_sum[i]
        Normalized_data <- cbind(Normalized_data, x)
    }
    colnames(Normalized_data) <- names(sample_sum)

} else return(Final_matrix)
  
print(paste('No.of NA values in Normalized data:',sum(is.na(Normalized_data)== TRUE)))

In [ ]:
write.csv(Normalized_data,file.path(fName,'Normalised_Quant_table.csv'),row.names =TRUE) 